## Lets scrape Biggest collection of Movies and Create dataset of 14285 movies dataset
### Bollywood Movies
#### Source : Imdb Website

In [1]:
# imports 
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd

In [2]:
base_url = 'https://www.imdb.com/search/title/?title_type=feature&primary_language=hi&sort=num_votes,desc&start={}&ref_=adv_nxt'
# create urls for searching all pages
urls = []
number = 1
while(number <14286):
    urls.append(base_url.format(number))
    number += 50

In [4]:
urls[100:104]
# len(urls) 286

['https://www.imdb.com/search/title/?title_type=feature&primary_language=hi&sort=num_votes,desc&start=5001&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=feature&primary_language=hi&sort=num_votes,desc&start=5051&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=feature&primary_language=hi&sort=num_votes,desc&start=5101&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=feature&primary_language=hi&sort=num_votes,desc&start=5151&ref_=adv_nxt']

In [5]:
# for certificate runtime and genre extraction
def get_movie_category(p):
    certificate = ''
    runtime = ''
    genre = ''
    if p.find('span',{'class':'certificate'}):
        certificate = p.find('span',{'class':'certificate'}).text.strip()
    if p.find('span',{'class':'runtime'}):
        runtime = p.find('span',{'class':'runtime'}).text.strip()
    if p.find('span',{'class':'genre'}):
        genre = p.find('span',{'class':'genre'}).text.strip()
    return (certificate,runtime,genre)

In [6]:
# Scrape all movie information from given card
def get_movie_info(card):
    try :
        movie ={}
        if card.find('h3'):
            movie_name = card.h3.text.split('.')
            movie["Movie Name"] = movie_name[1]
        if card.find('p'):
            for i,e in enumerate(get_movie_category(card.find('p'))):
                if i==0: movie['Certificate'] = e
                elif i==1: movie['RunTime'] = e  
                elif i==2: movie['Genre'] = e 
        if card.find('div',{'class':'ratings-bar'}):
            rating = card.find('div',{'class':'ratings-bar'}).find('div',{'class':'inline-block ratings-imdb-rating'}).strong.text
            movie['Rating'] = rating
        if card.find('p',{'class':'sort-num_votes-visible'}):
            p = card.find('p',{'class':'sort-num_votes-visible'})

            if p.find('span',{'name':'nv'}):
                    votes =''
                    votes = p.find('span',{'name':'nv'}).text
                    movie["Votes"] = votes
        if card.find('p',{'class':''}) :
            links = card.find('p',{'class':''}).find_all('a')
            stars = []
            for i,a in enumerate(links):
                if i==0:
                    movie['Director'] = a.text
                else : stars.append(a.text)
            movie["Stars"] = stars
        return movie
    except : return None

In [8]:
# Scrape all movies from given page by passing page of url
def get_all_movies_from_web_page(url,movies):
    web_page_content = req.get(url,headers={"User-Agent": "Requests"}).content
    # convert into beautifulsoup object
    bsoup = bs(web_page_content)
    # important content 
    card_blocks = bsoup.find_all('div',{'class':'lister-item-content'})
    for card in card_blocks:
        movie = get_movie_info(card)
        if movie != None :
            movies.append(get_movie_info(card))

In [52]:
# Lets pass all the souce url collected from imdb website to the above function i.e. get_all_movies_from_web_page
movies = []
# this urls contains all disney movies page 1 to page 286
for i,url in enumerate(urls):
    get_all_movies_from_web_page(url,movies)


In [51]:
movies = []
# scraping all movies from pages urls
for url in urls:
    get_all_movies_from_web_page(url,movies)

In [58]:
# Lets format our data to objects to lists for feeding into pandas
def format_data(movies,keys):
    movies_name = []
    certificates =[]
    run_times = []
    genres = []
    ratings = []
    directors = []
    stars =[]
    votes =[]
    for movie in movies:
        for key in keys:
            if key == 'Stars' :
                if key in movie.keys():
                    stars_string = ""
                    for star in movie["Stars"]:
                        stars_string = stars_string + star +","
                    stars.append(stars_string)
                else : stars.append('None')
            elif key == "Movie Name":
                if key in movie.keys():
                    name_filter = movie[key].strip()
                    movies_name.append(name_filter.replace("\n"," "))
                else : movies_name.append('None')
            elif key == "Certificate":
                if key in movie.keys():
                    certificates.append(movie[key])
                else : certificates.append('None')
            elif key == "RunTime":
                if key in movie.keys():
                    run_times.append(movie[key])
                else : run_times.append('None')
            elif key == "Genre":
                if key in movie.keys():
                    genres.append(movie[key])
                else : genres.append('None')
            elif key == "Rating":
                if key in movie.keys():
                    ratings.append(movie[key])
                else : ratings.append('None')
            elif key == "Director":
                if key in movie.keys():
                    directors.append(movie[key])
                else : directors.append('None')
            elif key == "Votes":
                if key in movie.keys():
                    votes.append(movie[key])
                else : votes.append('None')
    return {"Movie Name":movies_name,"Imdb Ratings":ratings,"Genre":genres,"Run Time":run_times,"Director":directors,"Stars":stars,"Votes":votes,"Certificate":certificates}

In [59]:
object_keys = list(res[0].keys())
object_keys

['Movie Name',
 'Certificate',
 'RunTime',
 'Genre',
 'Rating',
 'Votes',
 'Director',
 'Stars']

In [64]:
movies_dictionary = format_data(res,object_keys)

In [65]:
len(movies_dictionary["Imdb Ratings"])

7434

In [68]:
data_frame = pd.DataFrame(movies_dictionary)
data_frame.tail()

,Movie Name,Imdb Ratings,Genre,Run Time,Director,Stars,Votes,Certificate
7429,Sanju (2018),7.7,"Biography, Comedy, Drama",155 min,Rajkumar Hirani,"Ranbir Kapoor,Paresh Rawal,Manisha Koirala,Vic...","47,535",UA
7430,Laxmii (2020),2.3,"Action, Comedy, Horror",141 min,Lawrence Raghavendra,"Balakrishnan Thevar,Akshay Kumar,Kiara Advani,...","46,454",15+
7431,Kabhi Khushi Kabhie Gham,7.4,"Drama, Musical, Romance",210 min,Karan Johar,"Shah Rukh Khan,Kajol,Amitabh Bachchan,Hrithik ...","45,026",U
7432,Rab Ne Bana Di Jodi (2008),7.2,"Comedy, Drama, Music",167 min,Aditya Chopra,"Shah Rukh Khan,Anushka Sharma,Vinay Pathak,M.K...","43,868",U
7433,Uri: The Surgical Strike (2018),8.2,"Action, Drama, War",138 min,Aditya Dhar,"Vicky Kaushal,Paresh Rawal,Mohit Raina,Yami Ga...","43,475",UA


In [69]:
# save dataframe into csv file
data_frame.to_csv('./data/bollywood-movies-dataset.csv')